# Final Project

Sections

In [14]:
import getpass
import pyspark
from pyspark.sql import SparkSession

from pyspark.sql import functions as F
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType
from pandas import Series
import datetime
import time

import pandas as pd
import numpy as np

from math import sin, cos, sqrt, atan2, radians

# Spark session

In [2]:
spark = (SparkSession
         .builder
         .appName('final-proj')
         .master('yarn')
         .config('spark.port.maxRetries', 100)
         .config('spark.executor.memory', '1g')
         .config('spark.executor.instances', '2')
         .config('spark.executor.cores', '2')
         .config('spark.jars.packages', 'org.apache.spark:spark-streaming-kafka-0-8_2.11:2.2.0')
         .getOrCreate())

sc = spark.sparkContext
conf = sc.getConf()

spark

# Metadata

In [3]:
# Metadata file contains all the stations and gps coordinates
metadata = spark.read.text('/datasets/project/metadata/BFKOORD_GEO')

In [4]:
# Splitting and adding new columns to the spark dataframe
split_col = pyspark.sql.functions.split(metadata['value'], " % ")
split_left = pyspark.sql.functions.split(split_col.getItem(0), " +")
metadata = metadata.withColumn('longitude', split_left.getItem(2))
metadata = metadata.withColumn('latitude', split_left.getItem(1))
metadata = metadata.withColumn('stop', split_col.getItem(1))
metadata = metadata.drop('value')

metadataPandas = metadata.toPandas()
metadataPandas.head()

,longitude,latitude,stop
0,44.446770,26.074412,Bucuresti
1,50.901549,1.811446,Calais
2,51.284212,1.075329,Canterbury
3,50.729172,-3.543547,Exeter
4,46.922368,9.733756,"Fideris, Bahnhof"


In [5]:
# Zürich HB is the starting point
zurich_coord = metadataPandas[metadataPandas['stop'] == 'Zürich HB']
zurich_coord

,longitude,latitude,stop
2379,47.378177,8.540192,Zürich HB


In [6]:
# Function to calculate the distance of gps coordinates
def compute_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0 
    
    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)
    
    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance*1000

In [7]:
# keep only stations inside 10km
mask = metadataPandas.apply(lambda x: compute_distance(
    float(zurich_coord['latitude']), 
    float(zurich_coord['longitude']),
    float(x['latitude']), 
    float(x['longitude'])) <= 10000, axis=1)
metadataPandas = metadataPandas[mask]

# Source Data

In [8]:
# Directory with the source data
df = spark.read.option("delimiter", ";").option("header", "true").csv('/datasets/project/istdaten/*/*')

In [9]:
# Keep only the needed information
df = df.select(
          df['BETRIEBSTAG'].alias('date'), 
          df['FAHRT_BEZEICHNER'].alias('id'), 
          df['PRODUKT_ID'].alias('transport_type'), 
          df['LINIEN_ID'].alias('train_number'), 
          df['HALTESTELLEN_NAME'].alias('stop_name'), 
          df['ANKUNFTSZEIT'].alias('arrival_time'), 
          df['AN_PROGNOSE'].alias('actual_arrival_time'),
          df['AN_PROGNOSE_STATUS'].alias('status_arrival_time'),
          df['ABFAHRTSZEIT'].alias('departure_time'),
          df['AB_PROGNOSE'].alias('actual_departure_time'),
          df['AB_PROGNOSE_STATUS'].alias('status_departure_time'),
          df['DURCHFAHRT_TF'].alias('stop_here'))

In [10]:
# compute the departure and arrival delays
formatTS1 = "dd.MM.yyyy HH:mm:ss"
formatTS2 = "dd.MM.yyyy HH:mm"

departure_delay = (F.unix_timestamp('actual_departure_time', formatTS1) - 
        F.unix_timestamp('departure_time', formatTS2))
arrival_delay = (F.unix_timestamp('actual_arrival_time', formatTS1) - 
        F.unix_timestamp('arrival_time', formatTS2))

# extract the day of the week
def get_week_day(date):
    converted_date = datetime.datetime.strptime(str(date), "%d.%m.%Y")
    return converted_date.weekday()

week_day_udf = F.udf(get_week_day, IntegerType())

# udf to fill NaN departure (arrival) values with the arrival (departure) time
@udf('string')
def fillWithOther(a, b):
    if (a is None):
        return b
    return a
    
# date parser
date = F.to_date('date', 'dd.MM.yyyy')

# First filtering. arrival_delay and departure_delay columns
df_filtered = (df.filter((df.status_arrival_time == 'GESCHAETZT') | (df.status_departure_time == 'GESCHAETZT'))
     .withColumn("arrival_delay", arrival_delay)
     .withColumn("departure_delay", departure_delay)
                      )

# Handling the NaN values
df_with_delays = df_filtered.select('id', 'train_number', 'stop_name', 'arrival_time', 'arrival_delay', 'departure_time', 'departure_delay')
df_with_delays = df_with_delays.fillna(0, ['departure_delay', 'arrival_delay'])
df_with_delays = (df_with_delays
                  .withColumn('arrival_time2', fillWithOther(df_with_delays.arrival_time, df_with_delays.departure_time))
                  .withColumn('departure_time2', fillWithOther(df_with_delays.departure_time, df_with_delays.arrival_time))
                  .drop(df_with_delays.departure_time).drop(df_with_delays.arrival_time)
                 )
df_with_delays = df_with_delays.withColumnRenamed('departure_time2', 'departure_time').withColumnRenamed('arrival_time2', 'arrival_time')
# Adding the day of the week
df_with_delays = df_with_delays.withColumn("day_week", week_day_udf(F.split(df_with_delays.arrival_time, " ")[0]))

# Filtering outliers
df_with_delays = df_with_delays.filter((df_with_delays['arrival_delay'] > -1000) &
                                       (df_with_delays['arrival_delay'] < 1000) &
                                       (df_with_delays['departure_delay'] > -1000) &
                                       (df_with_delays['departure_delay'] < 1000)
                                      )

In [11]:
# Filtering the stops that are not in the metadata
# Adding a new column 'info' containing all the needed information
# Grouping by trip id
# Collect the list of the 'info' rows in the group
df_grouped = (df_with_delays
    .filter(df_with_delays.stop_name.isin(metadataPandas.stop.tolist()))
    .withColumn('info', F.struct('stop_name', F.split('arrival_time', " ")[1], 'arrival_delay', F.split('departure_time', " ")[1], 'departure_delay', 'day_week'))
    .groupby('id')
    .agg(F.collect_list('info').alias('info'))
              )

In [12]:
# Collecting the groups
groups = df_grouped.collect()

In [57]:
# Function to compute the delta between departur and arrival time. If the delta is too high, it will be filtered
def compute_delta(arrival, departure, day):
    d1 = datetime.datetime.strptime(arrival, "%H:%M")
    d2 = datetime.datetime.strptime(departure, "%H:%M")
    d1_ts = time.mktime(d1.timetuple())
    d2_ts = time.mktime(d2.timetuple())
    res = int(d2_ts-d1_ts) / 60
    if (day > 0):
        res = res + 60 * 24 * day
    return res

def delays(s):
    return tuple(s)
    if s.shape[0] < 2:
        return s
    return list(s)

# DataFrame containing the final schedule
schedule = pd.DataFrame(columns= ['trip_id', 'dep_node', 'arr_node', 
                                  'dep_time', 'arr_time',
                                  'mean_delay', 'std_delay', 
                                  'dep_day', 'arr_day', 'delta', 'delays'])

# Iterating through groups
for group in groups:
    trip_id = group[0]
    info = group[1]
    p = pd.DataFrame(info, columns=['stop', 'arrival_time', 'arrival_delay','departure_time', 'departure_delay', 'week_day'])
    
    # Computing the mean of arrival and departure delays
    c = p.groupby(['week_day', 'arrival_time', 'departure_time', 'stop']).agg([np.mean, np.std, delays])

    c = c.reset_index()
    c.columns = [' '.join(col).strip() for col in c.columns.values]
    
    # Creating all the couples of (start_station, arrival_station) for each trip. The order of the schedule 
    # is manteined by the groupby operation. We can have the same schedule for different days of the week, 
    # or the schedule can even change on different days. The compute_delta function calculates the time interval
    # between each couple: if the interval is too high, it means that a new trip started and the couple has to be
    # discarded. We also consider trips that are scheduled between midnight (in this case the week_day changes). 
    partialResult = pd.DataFrame([(trip_id,
                        c.loc[i, 'stop'], 
                        c.loc[i+1, 'stop'], 
                        c.loc[i, 'departure_time'], 
                        c.loc[i+1, 'arrival_time'], 
                        c.loc[i+1, 'arrival_delay mean'],
                        c.loc[i+1, 'arrival_delay std'], 
                        c.loc[i, 'week_day'], 
                        c.loc[i+1, 'week_day'], 
                        compute_delta(c.loc[i, 'departure_time'], 
                                      c.loc[i+1, 'arrival_time'], 
                                      c.loc[i+1, 'week_day']-c.loc[i, 'week_day']),
                        c.loc[i+1, 'arrival_delay delays']) 
                                  for i in range(len(c)-1)],
                     columns= ['trip_id', 'dep_node', 'arr_node', 
                               'dep_time', 'arr_time',
                               'mean_delay', 'std_delay', 
                               'dep_day', 'arr_day', 'delta', 'delays'])
    # Need to merge the last station with the first, in case of a trip that starts on sunday night and finisches
    # on monday morning
    m = len(c)-1
    partialResult = partialResult.append([{'trip_id': trip_id,
                'dep_node': c.loc[m, 'stop'], 
                'arr_node': c.loc[0, 'stop'], 
                'dep_time': c.loc[m, 'departure_time'], 
                'arr_time': c.loc[0, 'arrival_time'],
                'mean_delay': c.loc[m, 'arrival_delay mean'], 
                'std_delay': c.loc[m, 'arrival_delay std'], 
                'dep_day': c.loc[m, 'week_day'], 
                'arr_day': c.loc[0, 'week_day'],
                'delta': compute_delta(c.loc[m, 'departure_time'], c.loc[0, 'arrival_time'], c.loc[0, 'week_day']+7-c.loc[m, 'week_day']),
                'delays': c.loc[m, 'arrival_delay delays']
                 }], ignore_index=True
               )
    # Filtering the wrong couples and couples where the start and stop stations are the same
    partialResult = partialResult[partialResult.delta < 180]
    partialResult = partialResult[partialResult.dep_node != partialResult.arr_node]
    #partialResult = partialResult.drop(['delta'], axis=1)
    schedule = schedule.append(partialResult)

schedule.reset_index(drop=True, inplace=True)
schedule.head()

,trip_id,dep_node,arr_node,dep_time,arr_time,mean_delay,std_delay,dep_day,arr_day,delta,delays
0,85:11:18388:001,Dietlikon,Stettbach,23:14,23:17,60.151515,105.014023,0,0,3.0,"(-2, 341, 194, 346, -71, 70, -5, 41, 3, 1, -20..."
1,85:11:18388:001,Stettbach,Zürich Stadelhofen,23:18,23:22,67.666667,97.935459,0,0,4.0,"(1, 316, 187, 406, -1, 37, -1, 18, 12, 9, 13, ..."
2,85:11:18388:001,Zürich Stadelhofen,Zürich HB,23:23,23:26,14.606061,92.191695,0,0,3.0,"(-45, 221, 125, 354, -46, -25, -40, -39, -38, ..."
3,85:11:18388:001,Zürich HB,Zürich Hardbrücke,23:29,23:31,40.606061,38.486150,0,0,2.0,"(38, 97, 47, 234, 19, 43, 29, 25, 40, 37, 23, ..."
4,85:11:18388:001,Zürich Hardbrücke,Zürich Altstetten,23:31,23:35,3.545455,40.070322,0,0,4.0,"(-22, 27, 18, 174, -34, 1, -22, -22, -7, 7, -1..."


In [58]:
# We found some wrong schedules, always between Zurich HB and Zurich Oerlikon. 
# We suppose that these are errors, because in other days the same schedule doesn't include Zurich Oerlikon
schedule[(schedule['dep_time'] > schedule['arr_time']) & (schedule['dep_day'] == schedule['arr_day'])].head()

,trip_id,dep_node,arr_node,dep_time,arr_time,mean_delay,std_delay,dep_day,arr_day,delta,delays
1307,85:11:19256:001,Zürich HB,Zürich Oerlikon,15:14,15:12,-66.0,NaN,0,0,-2.0,"(-66,)"
4008,85:11:19254:001,Zürich HB,Zürich Oerlikon,14:44,14:42,-139.0,NaN,0,0,-2.0,"(-139,)"
15175,85:11:19253:001,Zürich HB,Zürich Oerlikon,14:17,14:16,54.0,NaN,0,0,-1.0,"(54,)"
15302,85:11:19285:001,Zürich HB,Zürich Oerlikon,22:17,22:16,156.0,NaN,3,3,-1.0,"(156,)"
19164,85:11:19247:001,Zürich HB,Zürich Oerlikon,12:47,12:46,94.0,NaN,0,0,-1.0,"(94,)"


In [60]:
# These are the trips that have to be modified
trip_id_to_modify = schedule[(schedule['dep_time'] > schedule['arr_time']) & (schedule['dep_day'] == schedule['arr_day'])]['trip_id']

# Modifying the wrong rows
for i, row in schedule.iterrows():
    if ((schedule.loc[i, 'trip_id'] in list(trip_id_to_modify.values)) &
       (schedule.loc[i, 'arr_node'] == 'Zürich Oerlikon')):
        schedule.loc[i, 'arr_node'] = schedule.loc[i+1, 'arr_node']
        schedule.loc[i, 'arr_time'] = schedule.loc[i+1, 'arr_time']
        schedule.loc[i, 'mean_delay'] = schedule.loc[i+1, 'mean_delay']
        schedule.loc[i, 'std_delay'] = schedule.loc[i+1, 'std_delay']
        
# Eliminating the wrong rows
schedule = schedule[ (~schedule['trip_id'].isin(list(trip_id_to_modify.values)))
        | ((schedule['trip_id'].isin(list(trip_id_to_modify.values)))
           & (schedule['dep_node'] != 'Zürich Oerlikon'  ) )]

In [61]:
# change time type and drop/rename columns
schedule = (schedule.drop(['delta'], axis=1))

def make_time(entry):
    #h, m = entry.split(':')
    return pd.Timedelta(entry+':00')

schedule['dep_time'] = schedule['dep_time'].apply(lambda x: make_time(x))
schedule['arr_time'] = schedule['arr_time'].apply(lambda x: make_time(x))
          
schedule.head()

,trip_id,dep_node,arr_node,dep_time,arr_time,mean_delay,std_delay,dep_day,arr_day,delays
0,85:11:18388:001,Dietlikon,Stettbach,23:14:00,23:17:00,60.151515,105.014023,0,0,"(-2, 341, 194, 346, -71, 70, -5, 41, 3, 1, -20..."
1,85:11:18388:001,Stettbach,Zürich Stadelhofen,23:18:00,23:22:00,67.666667,97.935459,0,0,"(1, 316, 187, 406, -1, 37, -1, 18, 12, 9, 13, ..."
2,85:11:18388:001,Zürich Stadelhofen,Zürich HB,23:23:00,23:26:00,14.606061,92.191695,0,0,"(-45, 221, 125, 354, -46, -25, -40, -39, -38, ..."
3,85:11:18388:001,Zürich HB,Zürich Hardbrücke,23:29:00,23:31:00,40.606061,38.486150,0,0,"(38, 97, 47, 234, 19, 43, 29, 25, 40, 37, 23, ..."
4,85:11:18388:001,Zürich Hardbrücke,Zürich Altstetten,23:31:00,23:35:00,3.545455,40.070322,0,0,"(-22, 27, 18, 174, -34, 1, -22, -22, -7, 7, -1..."


In [63]:
# Compute the walking times between stations. Keeping only stations within 300m
walk = pd.DataFrame(columns= ['trip_id', 'dep_node', 'arr_node', 
                              'dep_time', 'arr_time',
                              'mean_delay', 'std_delay', 
                              'dep_day', 'arr_day', 'delta', 'delays'])
for _, stop1 in metadataPandas.iterrows():
    for _, stop2 in metadataPandas.iterrows():
        if (stop1.stop != stop2.stop):
            distance = compute_distance(float(stop1.latitude), float(stop1.longitude), 
                                       float(stop2.latitude), float(stop2.longitude))
            if (distance <= 300):
                walk = walk.append({'trip_id': '0000', 
                                    'dep_node': stop1.stop, 
                                    'arr_node': stop2.stop, 
                                    'mean_delay': distance, # considering 1m/s
                                    'std_delay': 0.0}, ignore_index=True)

walk.head()

,trip_id,dep_node,arr_node,dep_time,arr_time,mean_delay,std_delay,dep_day,arr_day,delta,delays
0,0000,Bonstetten-Wettswil,"Bonstetten-Wettswil, Bahnhof",NaN,NaN,57.121084,0.0,NaN,NaN,NaN,NaN
1,0000,"Waldegg, Birmensdorferstrasse","Waldegg, Post",NaN,NaN,163.432216,0.0,NaN,NaN,NaN,NaN
2,0000,"Waldegg, Birmensdorferstrasse","Uitikon Waldegg, Bahnhof",NaN,NaN,287.582974,0.0,NaN,NaN,NaN,NaN
3,0000,"Zürich, Goldbrunnenplatz","Zürich, Zwinglihaus",NaN,NaN,262.946946,0.0,NaN,NaN,NaN,NaN
4,0000,Zürich HB,Zürich HB SZU,NaN,NaN,140.199392,0.0,NaN,NaN,NaN,NaN


In [64]:
schedule = schedule.append(walk)

In [65]:
schedule.to_pickle("data/schedule_raw.pkl")

In [66]:
schedule = pd.read_pickle("data/schedule_raw.pkl")

schedule.reset_index(drop=True, inplace=True)

# put the walking time in the arrival time
schedule.loc[schedule.trip_id=="0000", "walking_time"] = schedule.loc[schedule.trip_id == "0000", "mean_delay"].apply(lambda secs: pd.Timedelta("00:00:"+str(secs)))

# add column with the number of delays per edge
schedule.loc[~schedule.delays.isnull(), "num_delays"] = schedule.delays.loc[~schedule.delays.isnull()].apply(lambda x: np.NaN if x == np.NaN else len(x))

# sort columns and drop mean_delay and std_delay
schedule = schedule[["trip_id", "dep_node", "arr_node", "dep_day", "arr_day", "dep_time", "arr_time", "delays", "num_delays", "walking_time"]].copy()

schedule.head()

,trip_id,dep_node,arr_node,dep_day,arr_day,dep_time,arr_time,delays,num_delays,walking_time
0,85:11:18388:001,Dietlikon,Stettbach,0,0,0 days 23:14:00,0 days 23:17:00,"(-2, 341, 194, 346, -71, 70, -5, 41, 3, 1, -20...",33.0,NaT
1,85:11:18388:001,Stettbach,Zürich Stadelhofen,0,0,0 days 23:18:00,0 days 23:22:00,"(1, 316, 187, 406, -1, 37, -1, 18, 12, 9, 13, ...",33.0,NaT
2,85:11:18388:001,Zürich Stadelhofen,Zürich HB,0,0,0 days 23:23:00,0 days 23:26:00,"(-45, 221, 125, 354, -46, -25, -40, -39, -38, ...",33.0,NaT
3,85:11:18388:001,Zürich HB,Zürich Hardbrücke,0,0,0 days 23:29:00,0 days 23:31:00,"(38, 97, 47, 234, 19, 43, 29, 25, 40, 37, 23, ...",33.0,NaT
4,85:11:18388:001,Zürich Hardbrücke,Zürich Altstetten,0,0,0 days 23:31:00,0 days 23:35:00,"(-22, 27, 18, 174, -34, 1, -22, -22, -7, 7, -1...",33.0,NaT


- Drop edges with **few delays**.

Many edges happened **only few times** (we can see it  from the collected number of delays). To build our schedule we drop all the trips that happened **less times than a treshold** because if a trip took place 10 times over 33 weeks then it is reasonable to not consider it in the schedule. Notice that our schedule has been computed **week-wise** because we want to extract a schedule for each day of the week. Therefore, we won't have a different schedule for special days like Easter or Christmas.

In [ ]:
# separate bus/metro/train edges from walking edges
no_walk = schedule[(schedule.trip_id != "0000")]
walk = schedule[(schedule.trip_id == "0000")]

# verify how many null values are in the columns
# print("Null values in bus/metro/train edges:\n", no_walk.isnull().sum())
# print("\nNull values in walking edges:\n", walk.isnull().sum())

# check how many delays per bus/metro/train edge we have 
print("Number of delays per bus/metro/train edge:\n", no_walk.num_delays.describe())

In [ ]:
# and keep into the schedule only trips happened more than treshold times
treshold = 20
no_walk = no_walk[no_walk.num_delays >= treshold]
no_walk.head()

In [ ]:
# check again the number of delays. The schedule is now half as big and more meaningful
no_walk.num_delays.describe()

- **Clusterize** all the edges depending on their delays. Edges with similar delays will be associated with the same cluster and each cluster will be associated to a **gamma distribution** initialized with the mean and standart deviation of the delays of the whole cluster. 

We use KMeans to clusterize the edges, however, each edge has between 20 to 33 unordered delays so we extracted the `[20, 30, 40, 50, 60, 70, 80]` qth percentiles for each edge and used them as the features of that edge. This allows to represent more meaningfully the delay distribution of each edge while also dropping some possible outliers.

In [ ]:
percentiles = [20, 30, 40, 50, 60, 70, 80]
X = pd.DataFrame(
    columns=percentiles,
)
for p in percentiles:
    X[p] = no_walk.delays.apply(lambda delays: np.percentile(delays, p))
    
X.head()

Now that we have the features we can run the **KMeans**. Note that the index is the same as the initial dataframe, i.e. we obviously know which percentiles belong to which edge, therefore we can proceed in the clustering. We run KMeans for several number of clusters and plot their respective inertia, i.e. the **within-cluster sum-of-squares**. Then we use the elbow method to choose the more reasonable number of clusters.

In [ ]:
from sklearn.cluster import KMeans

n_clusters = [10, 50, 100, 200, 300, 500, 800]
# n_clusters = np.arange(10, 100, 20)

kmeans = {}
for n_cluster in n_clusters:
    print("Fitting on K="+str(n_cluster) + "...")
    model = KMeans(n_cluster, random_state=0, n_jobs=10)
    kmeans[n_cluster] = model
    
    model.fit(X)

plt.grid()
plt.ylabel("inertia")
plt.xlabel("K")
plt.plot(n_clusters, [model.inertia_ for _, model in kmeans.items()])

In [ ]:
# choose the best K and get the corresponding labels
K = 200
X['labels'] = kmeans[K].labels_
X.head()

In [ ]:
# plot a random subset of the clusters and use only the two percentiles 30 and 70 as dimensions
num_clusters_to_plt = 5
clusters = np.random.choice(K, num_clusters_to_plt, replace=False)
cluster_mapping = dict(zip(clusters, range(num_clusters_to_plt)))

small = X[X['labels'].isin(clusters)].copy()
small['labels'] = small['labels'].apply(lambda label: cluster_mapping[label]) # map the labels to an integer in [0, 1, ..., num_clusters_to_plt-1] so the color is automatically chosen well

plt.grid()
plt.scatter(small[30], small[70], c=small['labels'])

In [ ]:
schedule = pd.concat([no_walk, walk], axis=0)
schedule.loc[X.index, "label"] = X["labels"]

print(schedule.shape)
schedule.head()

In [ ]:
schedule.to_pickle("data/schedule_clustered.pkl")

## Fit a distribution to each cluster

We now create a **distribution** for each cluster and store them in the schedule.
Why?

- We have few samples per edge which may not be representative enough of its delay distribution.
- It is reasonable that some edges' delays are dependent. The clusters have hopefully grouped edges whose delays are jointly dependent.

In [ ]:
schedule = pd.read_pickle("data/schedule_clustered.pkl")
schedule.head(1)

- collect **all the delays** for each cluster

In [ ]:
# get the labels of the clusters
clusters_labels = sorted(schedule["label"].dropna().astype(int).unique())

# collect all the delays for each cluster
cluster_delays = {} # map cluster -> list fo delays
for c_id in clusters_labels:
    cluster = schedule[schedule["label"] == c_id]
    cluster_delays[c_id] = np.array([delay for edge_delays in cluster.delays for delay in edge_delays])

- **fit a gamma** for each cluster and plot corresponding distributions and histograms

In [ ]:
gammas = {} # map cluster label -> gamma distribution

# #clusters (K) = 200 = 5*40
ncols = 5
nrows = 40
fig, axs = plt.subplots(nrows, ncols, figsize=(4*ncols, 4*nrows))

axs = np.ndenumerate(axs)

# i = 0
for cl, delays in cluster_delays.items():
#     if i>=10:
#         break
#     i+=1
    
    _, ax = next(axs)
    
    sns.set(color_codes=True)

    # plot histogram
    ax.set(xlabel='Delay [s]')
    sns.distplot(delays, bins=100, kde=False, ax=ax, norm_hist=True)
    
    # fit the gamma on the delays of the cluster
    if cl in [70, 94]:
        # these two clusters are best fitted in this way
        gammas[cl] = gamma(
            *gamma.fit(
                delays,
                loc=delays.min()
            )
        )
    else:
        gammas[cl] = gamma(
            *gamma.fit(
                delays,
                floc=delays.min()-1e-10)
        )

    # plot the gamma's pdf on the same plot
    x = np.linspace(delays.min(),delays.max(),1000)
    ax.plot(x, gammas[cl].pdf(x), label='Gamma distribution')
    ax.set_title("Cluster:" + str(cl))
    
plt.tight_layout()

We can see that the **gammas fit reasonibly well** the delays of the clusters! Let's see the probabilities of the delays being **lower than 5 minutes** (300 seconds):

In [ ]:
pd.Series([gamma.cdf(300) for _, gamma in gammas.items()]).describe()

Most of the probabilities are high, it means that there is often a high probability that the transports do less than 5 minutes of delay and we can safely take a connection whose departure is scheduled 5 minutes after our arrival. 

- add a new column to the `schedule` to **store**, for each edge, **the distribution** of its cluster. Then, store the obtained `schedule`.

In [ ]:
for c_id in clusters_labels:
    schedule.loc[schedule["label"] == c_id, "distribution"] = gammas[c_id]
schedule.head(2)

In [ ]:
schedule.to_pickle("data/schedule_clustered.pkl")

## Final clean 
Add/remove the columns to obtain the final DataFrame that will be used by the `Planner`.

In [ ]:
schedule = pd.read_pickle("data/schedule_clustered.pkl")

# keep only columns used by the algorithm
schedule = schedule[["trip_id", "dep_node", "arr_node", "dep_day", "arr_day", "dep_time", "arr_time", "distribution", "walking_time"]]

# the prob column will store the probability of taking the edge, i.e. of taking a mean of transport at a specific time
schedule["prob"] = np.NaN
# the label column indicates the status of the edge
schedule["label"] = Status.Unvisited
# the prev_edge column indicates the previous edge taken on a path and will allow to reconstruct the whole path
schedule["prev_edge"] = np.NaN

schedule.head()

In [ ]:
schedule.to_pickle("data/schedule_clean.pkl")